In [0]:
# Parameters: JsonSource
dbutils.widgets.text("JsonSource", "")
dbutils.widgets.text("RawEntitlement", "")
dbutils.widgets.text("DataEntitlement", "")

In [0]:
# Source files
blob_account_name = "blob_account_name"
blob_container_name = "blob_container_name"
blob_relative_path =  dbutils.widgets.get("JsonSource")
blob_sas_token = dbutils.secrets.get("sas_token_name", "sastoken")

# Table & View
raw_entitlement = dbutils.widgets.get("RawEntitlement")
data_entitlement = dbutils.widgets.get("DataEntitlement")
entitlement_view = "entitlement_view"

if not blob_relative_path or not raw_entitlement or not data_entitlement:
    dbutils.notebook.exit("Idle")

In [0]:
wasbs_path = f"wasbs://{blob_container_name}@{blob_account_name}.blob.core.windows.net/{blob_relative_path}"
spark.conf.set(f"fs.azure.sas.{blob_container_name}.{blob_account_name}.blob.core.windows.net",blob_sas_token)

In [0]:
# Read data from Json
spark.read.json(wasbs_path).createOrReplaceTempView(entitlement_view)

In [0]:
# Write into raw table
production_columns = ", editionType" if blob_relative_path == "GetAll.json" else ""
spark.sql(f"""
    CREATE OR REPLACE TABLE {raw_entitlement}
    SELECT accountId, accountName, entitlements, productNames, sellableProductCode, subscriptionId, contractStart, contractEnd
    {production_columns}
    FROM entitlement_view
""")

In [0]:
# Forming Data for Entitlement
spark.sql(f"""
    SELECT subscriptionId, accountId, accountName, sellableProductCode, 
    EXPLODE(entitlements) AS entitlements
    FROM {entitlement_view} 
""").createOrReplaceTempView(entitlement_view)

In [0]:
# Create and Save to Entitlement Data
spark.sql(f"""
    CREATE OR REPLACE TABLE {data_entitlement}
    SELECT subscriptionId, accountId, accountName, sellableProductCode,
    entitlements.startDate::timestamp AS startDate, entitlements.endDate::timestamp AS endDate, entitlements.name AS name, entitlements.metric AS metric, entitlements.quantity::DOUBLE AS quantity, entitlements.periodType AS periodType
    FROM {entitlement_view}
""")

In [0]:
dbutils.notebook.exit("OK")